# Try... SpectralNET Exploring Spatial Spectral Wavelet CNN for Hyper Spectral Image Classification


In [ ]:
'''
Important Links
https://paperswithcode.com/sota/hyperspectral-image-classification-on-kennedy
https://www.ehu.eus/ccwintco/index.php/Hyperspectral_Remote_Sensing_Scenes
https://github.com/tanmay-ty/SpectralNET
'''

'\nImportant Links\nhttps://paperswithcode.com/sota/hyperspectral-image-classification-on-kennedy\nhttps://www.ehu.eus/ccwintco/index.php/Hyperspectral_Remote_Sensing_Scenes\nhttps://github.com/tanmay-ty/SpectralNET\n'

### Installing required Modules 

In [ ]:
!pip install spectral
!pip install tenserflow
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212 kB 34.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tenserflow (from versions: none)
ERROR: No matching distribution found for tenserflow
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importing Required Modules

In [ ]:
import keras
import time
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from tensorflow.keras.models import Model
from keras.optimizers import Adam, SGD, Adagrad 
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import backend as Kb
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers import add, concatenate
from keras.layers import AveragePooling2D
from keras.utils import plot_model
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
 
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA
from operator import truediv

from plotly.offline import init_notebook_mode
 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

### GLOBAL VARIABLES

In [ ]:
dataset = 'KSC'
test_ratio = 0.8
windowSize = 24

### Data Preprocessing 

Define Functions

In [ ]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'data')
    if name == 'KSC': # Kennedy Space Center
        data = sio.loadmat('KSC.mat')['KSC']
        labels = sio.loadmat('KSC_gt.mat')['KSC_gt']
    elif name == 'BOT': # Botswana
        data = sio.loadmat(os.path.join(data_path, '/content/Botswana.mat'))['Botswana']
        labels = sio.loadmat(os.path.join(data_path, '/content/Botswana_gt.mat'))['Botswana_gt']
    elif name == 'CUP': # Cuprite
        data = sio.loadmat('CupriteS1_F224.mat')['Y']
        data = np.reshape(data, (250, 190, 224))
        labels = np.asarray(np.matrix(sio.loadmat('groundTruth_Cuprite_nEnd12.mat')['M'].argmax(1)))
        labels = np.transpose(labels)
        labels = np.reshape(labels, (224))
    elif name == 'URB': # Urban
        data = sio.loadmat('Urban_F210.mat')['Y']
        data = np.reshape(data, (307, 307, 210))
        # labels = sio.loadmat('end6_groundTruth.mat')['A']
        labels = np.asarray(np.matrix(sio.loadmat('end6_groundTruth.mat')['A'].argmax(0)))
        labels = np.reshape(labels, (307, 307))
    elif name == 'CHN':
        data = sio.loadmat('China_Change_Dataset.mat')['T2']
        labels = sio.loadmat('China_Change_Dataset.mat')['Multiple']
    elif name == 'USA':
        data = sio.loadmat('USA_Change_Dataset.mat')['T2']
        labels = sio.loadmat('USA_Change_Dataset.mat')['Multiple']

    return data, labels

In [ ]:
def applyFA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    fa = FactorAnalysis(n_components=numComponents, random_state=0)
    newX = fa.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, fa
    

In [ ]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [ ]:
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            # patchesLabels[patchIndex] = y[r-margin<224]
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

Main Function for Preprosessing

In [ ]:
X, y = loadData(dataset)

# print(X.shape, y.shape)
X.shape,y.shape
# X = np.reshape(X, (250, 190, 224))
# y = np.reshape(y, (224))
# print(X.shape, y.shape)

((512, 614, 176), (512, 614))

In [ ]:
print(np.unique(y))
num_labels = len(np.unique(y))-1
print(num_labels)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
13


In [ ]:
dict_k = {}
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        #if output_image[i][j] in [m for m in range(1,17)]:
        if (y[i][j] in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]):#, 15, 16]:
            if (y[i][j] not in dict_k):
                dict_k[y[i][j]]=0
            dict_k[y[i][j]] +=1
print(dict_k)

{8: 431, 12: 503, 13: 927, 11: 419, 5: 161, 1: 761, 4: 252, 6: 229, 2: 243, 3: 256, 10: 404, 7: 105, 9: 520}


Applying FA (Factor analysis) & PCA (Principal Component Analysis)

In [ ]:
K = 3
X_fa,fa = applyFA(X,numComponents=K)
print('X_fa', X_fa.shape)

X_pca, fa = applyPCA(X,numComponents=K)
print('X_pca', X_pca.shape)

X_fa (512, 614, 3)
X_pca (512, 614, 3)


In [ ]:
X_fa, y_1 = createImageCubes(X_fa, y, windowSize=windowSize)
X_fa.shape, y.shape

((5211, 24, 24, 3), (512, 614))

In [ ]:
X_pca, y_2 = createImageCubes(X_pca, y, windowSize=windowSize)
X_pca.shape, y.shape

((5211, 24, 24, 3), (512, 614))

In [ ]:
Xtrain_1, Xtest_1, ytrain_1, ytest_1 = splitTrainTestSet(X_fa, y_1, test_ratio)
print('FA', Xtrain_1.shape, Xtest_1.shape, ytrain_1.shape, ytest_1.shape)

FA (1042, 24, 24, 3) (4169, 24, 24, 3) (1042,) (4169,)


In [ ]:
Xtrain_2, Xtest_2, ytrain_2, ytest_2 = splitTrainTestSet(X_pca, y_2, test_ratio)
print('PCA', Xtrain_2.shape, Xtest_2.shape, ytrain_2.shape, ytest_2.shape)

PCA (1042, 24, 24, 3) (4169, 24, 24, 3) (1042,) (4169,)


In [ ]:
Xtrain_1 = Xtrain_1.reshape(-1, windowSize, windowSize, K, 1)
print('FA', Xtrain_1.shape)

Xtrain_2 = Xtrain_2.reshape(-1, windowSize, windowSize, K, 1)
print('PCA', Xtrain_2.shape)

FA (1042, 24, 24, 3, 1)
PCA (1042, 24, 24, 3, 1)


In [ ]:
ytrain_1 = np_utils.to_categorical(ytrain_1)
print('FA', ytrain_1.shape)

ytrain_2 = np_utils.to_categorical(ytrain_2)
print('PCA', ytrain_2.shape)

FA (1042, 13)
PCA (1042, 13)


### Model Building

Define Functions

In [ ]:
def WaveletTransformAxisY(batch_img):
    odd_img  = batch_img[:,0::2]
    even_img = batch_img[:,1::2]
    L = (odd_img + even_img) / 2.0
    H = Kb.abs(odd_img - even_img)
    return L, H

def WaveletTransformAxisX(batch_img):
    # transpose + fliplr
    tmp_batch = Kb.permute_dimensions(batch_img, [0, 2, 1])[:,:,::-1]
    _dst_L, _dst_H = WaveletTransformAxisY(tmp_batch)
    # transpose + flipud
    dst_L = Kb.permute_dimensions(_dst_L, [0, 2, 1])[:,::-1,...]
    dst_H = Kb.permute_dimensions(_dst_H, [0, 2, 1])[:,::-1,...]
    return dst_L, dst_H

In [ ]:
def Wavelet(batch_image):
    # make channel first image
    batch_image = Kb.permute_dimensions(batch_image, [0, 3, 1, 2])
    r = batch_image[:,0]
    g = batch_image[:,1]
    b = batch_image[:,2]

    # level 1 decomposition
    wavelet_L, wavelet_H = WaveletTransformAxisY(r)
    r_wavelet_LL, r_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    r_wavelet_HL, r_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_L, wavelet_H = WaveletTransformAxisY(g)
    g_wavelet_LL, g_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    g_wavelet_HL, g_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_L, wavelet_H = WaveletTransformAxisY(b)
    b_wavelet_LL, b_wavelet_LH = WaveletTransformAxisX(wavelet_L)
    b_wavelet_HL, b_wavelet_HH = WaveletTransformAxisX(wavelet_H)

    wavelet_data = [r_wavelet_LL, r_wavelet_LH, r_wavelet_HL, r_wavelet_HH, 
                    g_wavelet_LL, g_wavelet_LH, g_wavelet_HL, g_wavelet_HH,
                    b_wavelet_LL, b_wavelet_LH, b_wavelet_HL, b_wavelet_HH]
    transform_batch = Kb.stack(wavelet_data, axis=1)

    # level 2 decomposition
    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(r_wavelet_LL)
    r_wavelet_LL2, r_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    r_wavelet_HL2, r_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)

    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(g_wavelet_LL)
    g_wavelet_LL2, g_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    g_wavelet_HL2, g_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)

    wavelet_L2, wavelet_H2 = WaveletTransformAxisY(b_wavelet_LL)
    b_wavelet_LL2, b_wavelet_LH2 = WaveletTransformAxisX(wavelet_L2)
    b_wavelet_HL2, b_wavelet_HH2 = WaveletTransformAxisX(wavelet_H2)


    wavelet_data_l2 = [r_wavelet_LL2, r_wavelet_LH2, r_wavelet_HL2, r_wavelet_HH2, 
                    g_wavelet_LL2, g_wavelet_LH2, g_wavelet_HL2, g_wavelet_HH2,
                    b_wavelet_LL2, b_wavelet_LH2, b_wavelet_HL2, b_wavelet_HH2]
    transform_batch_l2 = Kb.stack(wavelet_data_l2, axis=1)

    # level 3 decomposition
    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(r_wavelet_LL2)
    r_wavelet_LL3, r_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    r_wavelet_HL3, r_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(g_wavelet_LL2)
    g_wavelet_LL3, g_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    g_wavelet_HL3, g_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(b_wavelet_LL2)
    b_wavelet_LL3, b_wavelet_LH3 = WaveletTransformAxisX(wavelet_L3)
    b_wavelet_HL3, b_wavelet_HH3 = WaveletTransformAxisX(wavelet_H3)

    wavelet_data_l3 = [r_wavelet_LL3, r_wavelet_LH3, r_wavelet_HL3, r_wavelet_HH3, 
                    g_wavelet_LL3, g_wavelet_LH3, g_wavelet_HL3, g_wavelet_HH3,
                    b_wavelet_LL3, b_wavelet_LH3, b_wavelet_HL3, b_wavelet_HH3]
    transform_batch_l3 = Kb.stack(wavelet_data_l3, axis=1)

    # level 4 decomposition
    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(r_wavelet_LL3)
    r_wavelet_LL4, r_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    r_wavelet_HL4, r_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)

    wavelet_L4, wavelet_H4 = WaveletTransformAxisY(g_wavelet_LL3)
    g_wavelet_LL4, g_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    g_wavelet_HL4, g_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)

    wavelet_L3, wavelet_H3 = WaveletTransformAxisY(b_wavelet_LL3)
    b_wavelet_LL4, b_wavelet_LH4 = WaveletTransformAxisX(wavelet_L4)
    b_wavelet_HL4, b_wavelet_HH4 = WaveletTransformAxisX(wavelet_H4)


    wavelet_data_l4 = [r_wavelet_LL4, r_wavelet_LH4, r_wavelet_HL4, r_wavelet_HH4, 
                    g_wavelet_LL4, g_wavelet_LH4, g_wavelet_HL4, g_wavelet_HH4,
                    b_wavelet_LL4, b_wavelet_LH4, b_wavelet_HL4, b_wavelet_HH4]
    transform_batch_l4 = Kb.stack(wavelet_data_l4, axis=1)

    # print('shape before')
    # print(transform_batch.shape)
    # print(transform_batch_l2.shape)
    # print(transform_batch_l3.shape)
    # print(transform_batch_l4.shape)

    decom_level_1 = Kb.permute_dimensions(transform_batch, [0, 2, 3, 1])
    decom_level_2 = Kb.permute_dimensions(transform_batch_l2, [0, 2, 3, 1])
    decom_level_3 = Kb.permute_dimensions(transform_batch_l3, [0, 2, 3, 1])
    decom_level_4 = Kb.permute_dimensions(transform_batch_l4, [0, 2, 3, 1])
    
    # print('shape after')
    # print(decom_level_1.shape)
    # print(decom_level_2.shape)
    # print(decom_level_3.shape)
    # print(decom_level_4.shape)
    return [decom_level_1, decom_level_2, decom_level_3, decom_level_4]


def Wavelet_out_shape(input_shapes):
    # print('in to shape')
    return [tuple([None, 112, 112, 12]), tuple([None, 56, 56, 12]), 
            tuple([None, 28, 28, 12]), tuple([None, 14, 14, 12])]

Main Function for Model Building

In [ ]:
S1 = windowSize
L1 = K
#cuprite =12, Urban=6, KSC= 13, Botswanna=14
output_units = 13

In [ ]:
img_batch = Kb.zeros(shape=(8, 24, 24, 3), dtype='float32')
Wavelet(img_batch)

[<tf.Tensor: shape=(8, 12, 12, 12), dtype=float32, numpy=
 array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          .

In [ ]:
def get_wavelet_cnn_model():
 
    input_shape =  24, 24, 3
 
    input_ = Input(input_shape, name='the_input')
    # wavelet = Lambda(Wavelet, name='wavelet')
    wavelet = Lambda(Wavelet, Wavelet_out_shape, name='wavelet')
    input_l1, input_l2, input_l3, input_l4 = wavelet(input_)
    # print(input_l1)
    # print(input_l2)
    # print(input_l3)
    # print(input_l4)
    # level one decomposition starts
    conv_1 = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_1')(input_l1)
    norm_1 = BatchNormalization(name='norm_1')(conv_1)
    relu_1 = Activation('relu', name='relu_1')(norm_1)
 
    conv_1_2 = Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_1_2')(relu_1)
    norm_1_2 = BatchNormalization(name='norm_1_2')(conv_1_2)
    relu_1_2 = Activation('relu', name='relu_1_2')(norm_1_2)
 
    # level two decomposition starts
    conv_a = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_a')(input_l2)
    norm_a = BatchNormalization(name='norm_a')(conv_a)
    relu_a = Activation('relu', name='relu_a')(norm_a)
 
    # concate level one and level two decomposition
    concate_level_2 = concatenate([relu_1_2, relu_a])
    conv_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_2')(concate_level_2)
    norm_2 = BatchNormalization(name='norm_2')(conv_2)
    relu_2 = Activation('relu', name='relu_2')(norm_2)
 
    conv_2_2 = Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_2_2')(relu_2)
    norm_2_2 = BatchNormalization(name='norm_2_2')(conv_2_2)
    relu_2_2 = Activation('relu', name='relu_2_2')(norm_2_2)
 
    # level three decomposition starts 
    conv_b = Conv2D(filters=64, kernel_size=(3, 3), padding='same', name='conv_b')(input_l3)
    norm_b = BatchNormalization(name='norm_b')(conv_b)
    relu_b = Activation('relu', name='relu_b')(norm_b)
 
    conv_b_2 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_b_2')(relu_b)
    norm_b_2 = BatchNormalization(name='norm_b_2')(conv_b_2)
    relu_b_2 = Activation('relu', name='relu_b_2')(norm_b_2)
 
    # concate level two and level three decomposition 
    concate_level_3 = concatenate([relu_2_2, relu_b_2])
    conv_3 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_3')(concate_level_3)
    norm_3 = BatchNormalization(name='nomr_3')(conv_3)
    relu_3 = Activation('relu', name='relu_3')(norm_3)
 
    conv_3_2 = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_3_2')(relu_3)
    norm_3_2 = BatchNormalization(name='norm_3_2')(conv_3_2)
    relu_3_2 = Activation('relu', name='relu_3_2')(norm_3_2)
 
    # level four decomposition start
    conv_c = Conv2D(64, kernel_size=(3, 3), padding='same', name='conv_c')(input_l4)
    norm_c = BatchNormalization(name='norm_c')(conv_c)
    relu_c = Activation('relu', name='relu_c')(norm_c)
 
    conv_c_2 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_c_2')(relu_c)
    norm_c_2 = BatchNormalization(name='norm_c_2')(conv_c_2)
    relu_c_2 = Activation('relu', name='relu_c_2')(norm_c_2)
 
    conv_c_3 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_c_3')(relu_c_2)
    norm_c_3 = BatchNormalization(name='norm_c_3')(conv_c_3)
    relu_c_3 = Activation('relu', name='relu_c_3')(norm_c_3)
 
    # concate level level three and level four decomposition
    concate_level_4 = concatenate([relu_3_2, relu_c_3])
    conv_4 = Conv2D(256, kernel_size=(3, 3), padding='same', name='conv_4')(concate_level_4)
    norm_4 = BatchNormalization(name='norm_4')(conv_4)
    relu_4 = Activation('relu', name='relu_4')(norm_4)
 
    conv_4_2 = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', name='conv_4_2')(relu_4)
    norm_4_2 = BatchNormalization(name='norm_4_2')(conv_4_2)
    relu_4_2 = Activation('relu', name='relu_4_2')(norm_4_2)
 
    conv_5_1 = Conv2D(128, kernel_size=(3, 3), padding='same', name='conv_5_1')(relu_4_2)
    norm_5_1 = BatchNormalization(name='norm_5_1')(conv_5_1)
    relu_5_1 = Activation('relu', name='relu_5_1')(norm_5_1)
 
    pool_5_1 = AveragePooling2D(pool_size=(7, 7), strides=1, padding='same', name='avg_pool_5_1')(relu_5_1)
    #flat_5_1 = Flatten(name='flat_5_1')(pool_5_1) 
 
    #fc_5 = Dense(2048, name='fc_5')(flat_5_1)
    #norm_5 = BatchNormalization(name='norm_5')(fc_5)
    #relu_5 = Activation('relu', name='relu_5')(norm_5)
    #drop_5 = Dropout(0.5, name='drop_5')(relu_5)
 
    #fc_6 = Dense(2048, name='fc_6')(drop_5)
    #norm_6 = BatchNormalization(name='norm_6')(fc_6)
    #relu_6 = Activation('relu', name='relu_6')(norm_6)
    #drop_6 = Dropout(0.5, name='drop_6')(relu_6)
    flatten_layer = Flatten()(pool_5_1)
 
    dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    dense_layer2 = Dense(units=1024, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)
 
    model = Model(inputs=input_, outputs=output_layer)
    model.summary()
    plot_model(model, to_file='wavelet_cnn_0.5.png')
 
    return model

In [ ]:
model_1 = get_wavelet_cnn_model()
model_2 = get_wavelet_cnn_model()
model_3 = get_wavelet_cnn_model()
model_4 = get_wavelet_cnn_model()
model_5 = get_wavelet_cnn_model()
model_6 = get_wavelet_cnn_model()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 the_input (InputLayer)         [(None, 24, 24, 3)]  0           []                               
                                                                                                  
 wavelet (Lambda)               [(None, 12, 12, 12)  0           ['the_input[0][0]']              
                                , (None, 6, 6, 12),                                               
                                 (None, 3, 3, 12),                                                
                                 (None, 2, 2, 12)]                                                
                                                                                                  
 conv_1 (Conv2D)                (None, 12, 12, 64)   6976        ['wavelet[0][0]']          

Compile model with SGD & Adam

In [ ]:
adam = Adam(learning_rate=0.02)
sgd = SGD(learning_rate=0.02, momentum=0.9, nesterov=False)
adagrad = Adagrad(learning_rate=0.02)
model_1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model_3.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model_2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_4.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model_5.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])
model_6.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])

Saving Model


In [ ]:
filepath = "model1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list_1 = [checkpoint]

filepath = "model2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list_2 = [checkpoint]

filepath = "model3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list_3 = [checkpoint]

filepath = "model4.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list_4 = [checkpoint]

filepath = "model5.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list_5 = [checkpoint]

filepath = "model6.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list_6 = [checkpoint]

In [ ]:
# SGD & FA
history_1 = model_1.fit(x=Xtrain_1, y=ytrain_1, batch_size = 30, epochs=115 , callbacks=callbacks_list_1)

# SGD & PCA
history_2 = model_1.fit(x=Xtrain_2, y=ytrain_2, batch_size = 30, epochs=115 , callbacks=callbacks_list_2)

# Adam & FA
history_3 = model_2.fit(x=Xtrain_1, y=ytrain_1, batch_size = 30, epochs=115 , callbacks=callbacks_list_3)

# Adam & PCA
history_4 = model_2.fit(x=Xtrain_2, y=ytrain_2, batch_size = 30, epochs=115 , callbacks=callbacks_list_4)

# Adagrad & FA
history_5 = model_5.fit(x=Xtrain_1, y=ytrain_1, batch_size = 30, epochs=115 , callbacks=callbacks_list_5)

# Adagrad & PCA
history_6 = model_5.fit(x=Xtrain_2, y=ytrain_2, batch_size = 30, epochs=115 , callbacks=callbacks_list_6)

Epoch 1/115
35/35 [==============================] - ETA: 0s - loss: 1.1052 - accuracy: 0.6449
Epoch 1: saving model to model1.hdf5
35/35 [==============================] - 17s 38ms/step - loss: 1.1052 - accuracy: 0.6449
Epoch 2/115
33/35 [===========================>..] - ETA: 0s - loss: 0.4089 - accuracy: 0.8626
Epoch 2: saving model to model1.hdf5
35/35 [==============================] - 1s 22ms/step - loss: 0.4050 - accuracy: 0.8637
Epoch 3/115
34/35 [============================>.] - ETA: 0s - loss: 0.3682 - accuracy: 0.8912
Epoch 3: saving model to model1.hdf5
35/35 [==============================] - 1s 27ms/step - loss: 0.3682 - accuracy: 0.8906
Epoch 4/115
33/35 [===========================>..] - ETA: 0s - loss: 0.2953 - accuracy: 0.9182
Epoch 4: saving model to model1.hdf5
35/35 [==============================] - 1s 22ms/step - loss: 0.2847 - accuracy: 0.9213
Epoch 5/115
33/35 [===========================>..] - ETA: 0s - loss: 0.1659 - accuracy: 0.9485
Epoch 5: saving model to

SGD & FA

In [ ]:
fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(history_1.history['loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['loss'])
plt.title('Accuracy, Loss vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')

plt.show()

SGD & PCA

In [ ]:
fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(history_2.history['loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(history_2.history['accuracy'])
plt.plot(history_2.history['loss'])
plt.title('Accuracy, Loss vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')

plt.show()

Adam & FA

In [ ]:
fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(history_3.history['loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(history_3.history['accuracy'])
plt.plot(history_3.history['loss'])
plt.title('Accuracy, Loss vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')

plt.show()

Adam & PCA

In [ ]:
fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(history_4.history['loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(history_4.history['accuracy'])
plt.plot(history_4.history['loss'])
plt.title('Accuracy, Loss vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')

plt.show()

Adagrad & FA

In [ ]:
fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(history_5.history['loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(history_5.history['accuracy'])
plt.plot(history_5.history['loss'])
plt.title('Accuracy, Loss vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')

plt.show()

Adagrad & PCA

In [ ]:
fig = plt.figure(figsize=(12, 5))

fig.add_subplot(121)

plt.plot(history_6.history['loss'])
plt.title('Loss vs. epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')

fig.add_subplot(122)

plt.plot(history_6.history['accuracy'])
plt.plot(history_6.history['loss'])
plt.title('Accuracy, Loss vs. epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper right')

plt.show()

In [ ]:
# load best weights
model_1.load_weights("model1.hdf5")
model_1.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model_3.load_weights("model2.hdf5")
model_3.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

Xtest_1 = Xtest_1.reshape(-1, windowSize, windowSize, K, 1)
print('FA', Xtest_1.shape)

Xtest_2 = Xtest_2.reshape(-1, windowSize, windowSize, K, 1)
print('PCA', Xtest_2.shape)

ytest_1 = np_utils.to_categorical(ytest_1)
print('FA', ytest_1.shape)

ytest_2 = np_utils.to_categorical(ytest_2)
print('PCA', ytest_2.shape)

In [ ]:
model_2.load_weights("model3.hdf5")
model_2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model_4.load_weights("model4.hdf5")
model_4.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model_5.load_weights("model5.hdf5")
model_5.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model_6.load_weights("model6.hdf5")
model_6.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Model Report

In [ ]:
# SGD & FA
Y_pred_test = model_1.predict(Xtest_1)
y_pred_test = np.argmax(Y_pred_test, axis=1)
 
classification = classification_report(np.argmax(ytest_1, axis=1), y_pred_test)
print(classification)

In [ ]:
# SGD & PCA
Y_pred_test = model_3.predict(Xtest_1)
y_pred_test = np.argmax(Y_pred_test, axis=1)
 
classification = classification_report(np.argmax(ytest_1, axis=1), y_pred_test)
print(classification)

In [ ]:
# Adam & FA
Y_pred_test = model_2.predict(Xtest_2)
y_pred_test = np.argmax(Y_pred_test, axis=1)
 
classification = classification_report(np.argmax(ytest_2, axis=1), y_pred_test)
print(classification)

In [ ]:
# Adam & PCA
Y_pred_test = model_4.predict(Xtest_2)
y_pred_test = np.argmax(Y_pred_test, axis=1)
 
classification = classification_report(np.argmax(ytest_2, axis=1), y_pred_test)
print(classification)

In [ ]:
# Adagrad & FA
Y_pred_test = model_5.predict(Xtest_2)
y_pred_test = np.argmax(Y_pred_test, axis=1)
 
classification = classification_report(np.argmax(ytest_2, axis=1), y_pred_test)
print(classification)

In [ ]:
# Adagrad & PCA
Y_pred_test = model_6.predict(Xtest_2)
y_pred_test = np.argmax(Y_pred_test, axis=1)
 
classification = classification_report(np.argmax(ytest_2, axis=1), y_pred_test)
print(classification)

So After Comparison Adagrad with PCA provide best results

### Prediction

In [ ]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [ ]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

In [ ]:
def reports (X_test,y_test,name):
    start = time.time()
    Y_pred = model_1.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()
    print(end - start)
    if name == 'KSC':
        target_names = ['Lagoon','Impounded water','Cattail marsh','CP savanna','Graminoid marsh','Mud flats',
                          'Salt marsh','Willow marsh','CP Hammock','Scrub','Oaks/CP','Oaks/Broadleaved','Barrow Pits']
    elif name == 'BOT':
        target_names = ['Lagoon','Impounded water','Cattail marsh','CP savanna','Graminoid marsh','Mud flats',
                          'Salt marsh','Willow marsh','CP Hammock','Scrub','Oaks/CP','Oaks/Broadleaved','Barrow Pits', 'Vinyard_vertical_trellis']
    elif name == 'CUP':
        target_names = ["Alunite", "Andradite", "Buddingtonite", "Dumortierite", "Kaolinite1", "Kaolinite2", "Muscovite", "Montmorillonite", "Nontronite", "Pyrope", "Sphene", "Chalcedony"]
    elif name == 'URB':
        target_name = [ "Asphalt", "Grass", "Tree", "Roof", "Metal","Dirt" ]
    elif name == 'CHN':
        target_name = ["soil","river","tree","building","road","agricultural field"]
    elif name == 'USA':
        target_name = ["soil","irrigated fields","river","building","type of cultivated land","grassland"]
        
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model_1.evaluate(X_test, y_test, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [ ]:
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(Xtest_1,ytest_1,dataset)

In [ ]:
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report.txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss (%)'.format(Test_loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

In [ ]:
X, y = loadData(dataset)

In [ ]:
height = y.shape[0]

width = y.shape[1]
PATCH_SIZE = windowSize

In [ ]:
K = 3
X,fa = applyFA(X, numComponents=K)

In [ ]:
X = padWithZeros(X, PATCH_SIZE//2)

In [ ]:
# calculate the predicted image
outputs = np.zeros((height,width))
for i in range(height):
    for j in range(width):
        target = int(y[i,j])
        if target == 0 :
            continue
        else :
            image_patch=Patch(X,i,j)
            X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2], 1).astype('float32')                                   
            prediction = (model_6.predict(X_test_image))
            prediction = np.argmax(prediction, axis=1)
            outputs[i][j] = prediction+1

In [ ]:
ground_truth = spectral.imshow(classes = y,figsize =(7,7))

In [ ]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))

In [ ]:
spectral.save_rgb("predictions.jpg", outputs.astype(int), colors=spectral.spy_colors)